In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

In [ ]:
df = pd.read_csv('data/tbrain1to5.csv')
train_Y = df['shop_tag'].map(lambda x : 49 if x == 'other' else int(x))
df = df.drop(['shop_tag'], axis=1)
df.head()

In [ ]:
df.dtypes

In [ ]:
df.describe()

In [ ]:
def Diff(li1, li2):
    return list(set(li1) - set(li2)) + list(set(li2) - set(li1))

In [ ]:
num_features = ['dt', 'txn_cnt', 'txn_amt', 'domestic_offline_cnt', 'domestic_online_cnt',
                'overseas_offline_cnt', 'overseas_online_cnt', 'domestic_offline_amt_pct', 'domestic_online_amt_pct', 
                'overseas_offline_amt_pct', 'overseas_online_amt_pct', 'card_1_txn_cnt', 
                'card_2_txn_cnt', 'card_3_txn_cnt', 'card_4_txn_cnt', 'card_5_txn_cnt', 'card_6_txn_cnt',
                'card_7_txn_cnt', 'card_8_txn_cnt', 'card_9_txn_cnt', 'card_10_txn_cnt', 'card_11_txn_cnt', 
                'card_12_txn_cnt', 'card_13_txn_cnt', 'card_14_txn_cnt', 'card_other_txn_cnt', 'card_1_txn_amt_pct', 'card_2_txn_amt_pct', 
                'card_3_txn_amt_pct', 'card_4_txn_amt_pct', 'card_5_txn_amt_pct', 'card_6_txn_amt_pct', 'card_7_txn_amt_pct', 'card_8_txn_amt_pct', 
                'card_9_txn_amt_pct', 'card_10_txn_amt_pct', 'card_11_txn_amt_pct', 'card_12_txn_amt_pct', 'card_13_txn_amt_pct', 'card_14_txn_amt_pct', 'slam']

object_features = Diff(df.columns, num_features)
object_features

In [ ]:
# check columns is null
df.isnull().sum()

In [ ]:
# 數值型遺失值
for c in num_features:
    non_na_df = df[c][~df[c].isnull()]
    mean_non_na = non_na_df.mean()
    
    # Nan replace with mean
    df[c][df[c].isnull()] = mean_non_na
    
    IQR = non_na_df.quantile(0.75) - non_na_df.quantile(0.25)
    upper = non_na_df.quantile(0.75) + IQR * 1.5
    lower = non_na_df.quantile(0.25) - IQR * 1.5
    df[c] = df[c].clip(lower, upper)

In [ ]:
# 類別型遺失值
for i in object_features:
    df[i] = df[i].fillna(df[i].median())

In [ ]:
df.isnull().sum()

In [ ]:
for c in object_features:
    df[c] = LabelEncoder().fit_transform(df[c])

In [ ]:
train_X = df
x_train, x_val, y_train, y_val = train_test_split(train_X, train_Y, test_size=0.1, random_state=10)
clf = GradientBoostingClassifier(n_estimators=200)
clf.fit(x_train, y_train)

In [ ]:
y_pred = clf.predict_proba(x_val)
print(np.sort(y_pred[0])[::-1])